# Report

## Introduction and data

### Subject Introduction

Der Vertrieb und Verkauf von Waren über das Internet ist zu einem wichtigen Vertriebskanal der modernen, digitalisierten Wirtschaft geworden. Diese Entwicklung wurde durch Covid19-Pandemie weiter verstärkt. So stiegen in Deutschland die im B2C E-Commerce erzielten Umsätze im Jahr 2020 gegenüber dem Vorjahr um fast 23% auf knapp 73 Milliarden Euro (Statista, 2022a). Auch innerhalb des B2B E-Commerce wird in den kommenden Jahren ein enormes Umsatzwachstum erwartet, in den USA wird der Umsatz im Jahr 2025 auf 2,5 Billionen steigen, dies entspricht einer Wachstumsrate von rund 11% p.a. (Statista, 2022b).

### Project Motivation

Während die Umsätze im E-Commerce Jahr für Jahr steigen, sind die Margen im Onlinehandel für einen Großteil der Händler marginal. Dennoch müssen Unternehmen hohe Investitionen in das Online-Geschäft tätigen, um Umsätze im stark umkämpften Online-Geschäft zu erzielen (Alvarez & Marsal, 2021). Damit sich diese nachhaltig für das Unternehmen lohnen, müssen langfristig verlässliche Gewinne erwirtschaftet werden. Dieses Ziel kann nur erreicht werden, wenn externe Risiken für das Unternehmen minimiert und intern strategisch richtige Entscheidungen zur Portfolioerweiterung getroffen werden. Im folgenden soll ein Datensatz, bestehend aus Transaktionsdaten eines E-Commerce Unternehmen aus dem Vereinigten Königreich analysiert werden, um Abhängigkeiten von bestimmten 1.) Ländern oder Regionen 2) Kunden 3) Produkten bzw. Produktgruppen zu identifizieren und 4) Potenziale für Umsatzwachstum / Gewinnsteigerung zu entdecken.

### General Research Question

Wie können Abhängigkeiten für den E-Commerce Shop langfristig reduziert werden und welche Maßnahmen können zur langfristigen Gewinnmaximierung getroffen werden?

### Data Description

Der Datensatz zeigt Transaktionsdaten eines E-Commerce Unternehmen aus dem Vereinigten Koenigreich. Die Daten wurden zwischen dem 01.12.2010 und dem 09.12.2011 erfasst.

Jeder Beobachtungswert stellt innerhalb des Datensatzes eine Transaktion dar, wobei mehreren Transaktionen eine Rechnung(snummer) zugeteilt werden kann.

Der Datensatz enthaelt in der ursprünglichen Version 541909 Beobachtungswerte und 8 Spalten.

Es wurden keine Informationen dazu bereitgestellt, wie die Daten erhoben wurden. Es wird davon ausgegangen, dass sie aus dem CRM oder E-Commerce Backend des Shops stammen.

### Data cleaning process

Um die Daten zu säubern und für die weitere Verarbeitung vorzubereiten, mussten einige Anpassungen vorgenommen werden. Neben den offensichlichen Anpassungen wie dem Entfernen von fehlenden Werten und Leerzeilen wurden Zeilen entfernt, in welchen die Produktnummer darauf hinwies, dass es sich nicht um eine relevante Transaktion, sondern beispielsweise um Testtransaktionen oder Postsendungen handelt. Außerdem wurde der Gesamtpreis pro Transaktion als weitere Variable hinzugefügt. Darüber hinaus wurden Zeilen ohne Kundennummer oder Beschreibung entfernt, da dies auf eine nicht korrekte Transaktion hinweist.

### Data Dictionary / Definition of key variables

| Spaltenname  | Beschreibung  | Skalenniveau | Format |
|---|---|---|---|
|InvoiceNo | Rechnungsnummer, 6-stellige Nummer, welche jeder Transaktion eindeutig zuordnet. <br>Falls die Nummer ein "c" enthält wurde die Transaktion storniert. | Nominal | object |
|StockCode | Produktnummer, 5-stellige Nummer, welche jedem einzelnem Produkt eindeutig zugeordnet ist. | Nominal | object |
|Description | Beschreibung des Produkts. | Nominal | object |
|Quantity | Menge der einzelnen Produkte pro Transaktion. | Verhaeltnis | int |
|InvoiceDate | Uhrzeit und Datum der Rechnung. | Intervall | date |
|UnitPrice | Preis pro Produkt. | Verhaeltnis | float |
|CustomerID | Kundennummer, 5-stellige Nummer, welche je einem Kunden eindeutig zugeordnet ist. | Nominal | object |
|Country | Name des Landes in welchem der Kunde ansässig ist. | Nominal | category |
|TotalPrice | Produkt aus Quantity und UnitPrice. | Verhaeltnis | float |

## Eda

In [1]:
# Bibliotheken importieren
import pandas as pd
import altair as alt
from pathlib import Path
import numpy as np
# CSV Einlesen 
parent_path = str(Path().resolve().parent) + "/"
data_path = "data/"
processed_path = "processed/"
processed_file = "processed_data.csv"
df = pd.read_csv(parent_path + data_path + processed_path + processed_file, encoding= 'unicode_escape')
# Weitere Korrekturen
# Doppelte Werte löschen
df = df.drop_duplicates()
# Fehlende Werte auslesen
df.isna().sum()
# Weitere fehlende Werte löschen
df = df.dropna(subset=['CustomerID'])
df = df.dropna(subset=['Description'])


#### H1: Die Einnahmen des Online-Shops werden maßgeblich vom Land beeinflusst, in dem das Unternehmen ansässig ist.

##### Chart Beschreibung
Um die Response Variable für die Hypothese H1 zu visualisieren, werden die im jeweiligen Land erzielten Umsätze für die Top 10 Ländern gezeigt. Als Visualisierungsart wird ein Säulendiagramm verwendet, da sich so die Verhältnismäßigkeiten gut erkennen lassen. Durch das Eliminieren der horizontalen Gridlines wird Clutter minimiert. Der Fokus wird durch die Sortierung der Balken von lang nach kurz auf das für die Hypothese relevante Hauptmerkmal gelenkt. Es wurde zugunsten des Designs ein aussagekräftiger Titel sowie Untertitel gewählt und das Design bewusst schlicht gehalten.

In [2]:
# Gesamtumsatz berechnen
gesamtumsatz = df['TotalPrice'].sum()
# Berechnung des Gesamtumsatzes für jedes Land
country_totals = df.groupby('Country')['TotalPrice'].sum()

# Berechnung des prozentualen Anteils am Gesamtumsatz für jedes Land
country_percentages = country_totals / gesamtumsatz * 100

# Erstellen eines DataFrame mit den Spalten Land und Anteil am Gesamtumsatz
df_country_percentages = pd.DataFrame({'Country': country_percentages.index, 'Percentage': country_percentages.values}).sort_values(by='Percentage', ascending=False).head(10) 

# Visualisierung des prozentualen Anteils am Gesamtumsatz für die top 10 Länder
alt.Chart(df_country_percentages).mark_bar().encode(
    x=alt.X('Country', sort='-y', axis=alt.Axis(title='Country',titleAnchor='start',grid=False)),
    y=alt.Y('Percentage',  axis=alt.Axis(title='Percentage of total turnover', labelAngle=0, titleAnchor='end',grid=False)),
    color=alt.condition(
        alt.FieldOneOfPredicate("Country", ["United Kingdom"]),
        alt.value("darkred"),
        alt.value("grey")
    ),
).properties(
    title='Revenue by Country',
    width=400,
    height=150
).configure_view(
    strokeWidth=0
).configure_title(
    fontSize=22,
    font="Arial",
    color="black",
    anchor="start"
).configure_axis(
    labelFont="Arial",
    titleFont="Arial",
    labelFontSize=14,
    titleFontSize=12,
    titleFontWeight="normal",
    labelColor="grey",
    titleColor="grey"
).properties(
    title={"text":["Turnover by country"], "subtitle":["How often orders were placed by country (percentage)"]},
)


alt.Chart(...)

#### H2: Die Wahrscheinlichkeit, dass ein Kunde ein Produkt zurückgibt, sinkt mit der Anzahl der getätigten Käufe.

##### Chart Beschreibung
Die Response Variable für Hypothese H3 (Rückgabewahrscheinlichkeit relativ zur Anzahl der getätigten Käufe je Kunde) wird durch ein Streudiagram visualisiert, da sich so die Beziehung zwischen den beiden Variablen visuell gut darstellen lässt. Clutter wurde wieder so weit wie möglich eliminiert. Hier ist ein Untertitel für das Verständnis nicht vonnöten. Der Titel ist für den Fokus des Betrachters ausreichend. Das Design ist schlicht und kommt mit wenigen Farben aus.

In [3]:
# Erstellen eines df, der Kunden nach ihrer CustomerID gruppiert Anzahl der Käufe und Rückgaben zählen
df_purchases_returns = df.groupby(['CustomerID']).agg({'InvoiceNo': 'count'})
df_purchases_returns['Total Returns'] = df[df['InvoiceNo'].str.contains('C')].groupby(['CustomerID'])['InvoiceNo'].count()

# Umbennenung
df_purchases_returns.rename(columns={'InvoiceNo': 'Total Purchases'}, inplace=True)

# Rückgabewahrscheinlichkeit berechnen
df_purchases_returns['Probability of Return'] = df_purchases_returns['Total Returns'] / df_purchases_returns['Total Purchases']

# Visualisierung der Beziehung zwischen der Anzahl der getätigten Käufe und der Rückgabewahrscheinlichkeit
chart = alt.Chart(df_purchases_returns).mark_circle().encode(
    x=alt.X('Total Purchases', axis=alt.Axis(title='Total purchases',grid=False,titleAnchor='start')),
    y=alt.Y('Probability of Return', axis=alt.Axis(title='Probability of return',grid=False,titleAnchor='end')),
    color=alt.condition(
        alt.FieldOneOfPredicate("Probability of Return", [""]),
        alt.value("darkred"),
        alt.value("grey")
    ),
    tooltip = ['Total Purchases', 'Probability of Return']
).configure_view(
    strokeWidth=0
).configure_title(
    fontSize=22,
    font="Arial",
    color="black",
    anchor="start"
).configure_axis(
    labelFont="Arial",
    titleFont="Arial",
    labelFontSize=14,
    titleFontSize=12,
    titleFontWeight="normal",
    labelColor="grey",
    titleColor="grey"
).properties(
    title={"text":["Return probability vs. number of purchases made"], "subtitle":[""]},
)

# Chart darstellen
chart

alt.Chart(...)

## Visualizations

### H1 Visualizations

In [4]:
df["InvoiceDate"]= pd.to_datetime(df["InvoiceDate"])
df = df[df["InvoiceDate"]< "12/1/2011 00:00"]

In [ ]:
# Filter nach den umsatzstärksten Ländern
df_plot = df.groupby("Country").sum().sort_values(["TotalPrice"], ascending=False)[:10]
df_plot["Country"] = df_plot.index
df_plot["Country"]["United Kingdom"] ="UK"
df_plot["Country"]["EIRE"] ="Ireland"
df_plot["TotalPrice"] = df_plot["TotalPrice"].div(1000)

#### Chart Beschreibung
Um zuerst eine Übersicht dem Zuhörer zu ermöglichen über den aktuellen Kenntnisstand werden die Umsätze nach Regionen / Ländern gezeigt. Hierfür werden die Regionen aufgrund der hohen Anzahl auf zehn begrenzt, um die Übersichtlichkeit beibehalten zu können. Als Visualisierungsart wird dabei ein Säulendiagramm verwendet, um die Verhältnisse der kategorialen (Land) und nummerischen Variable (Umsatz) zu visualisieren. Das Säulendiagramm wurde dabei gewählt, da hier sehr gut mehrere (absolute) Werte verglichen werden können.<br> Innerhalb der Visualisierung wird Clutter eleminiert indem Gridlinien, sowie die Rahmen rechts und oben entfernt werden (Closure). Außerdem wird der Fokus (im Kontext von Focus Attention) gezielt mit (hue) Farbe auf den Umsatz des Landes UK gelegt, da die Problematik auf der Verteilung und dem großen Anteil Uks liegt. Zudem wird im Kontext des Designs der Schwerpunkt auf ein schlichtes Design mit wenigen Farben (Schwarz, grau und dunkelrot) gelegt.

In [6]:

# Visualisierung der umsatzstärksten Kunden
chart_co1 = alt.Chart(df_plot).mark_bar().encode(
    y=alt.Y("TotalPrice",
            axis=alt.Axis(title="Revenue in (thousand) pound", 
                          labelAngle=0,
                          titleAnchor="end",
                          grid=False,
                          values=[1000,3000,5000,7000])),
    x=alt.X("Country",
            sort=["UK", "Netherlands","Ireland","France","Germany","Australia","Spain","Switzerland","Belgium","Sweden"],
            axis=alt.Axis(title = "Country",
                          titleAnchor="start",
                          labelAngle=0)),
    color=alt.condition(
        alt.FieldOneOfPredicate("Country", ["UK"]),
        alt.value("darkred"),     
        alt.value("grey")     
    )
).properties(
    title={"text":["Top ten countries with the highest sales"], "subtitle":["In the period from 01.12.2010 to 30.11.2011"]},
    width=650,
    height=350
)


alt.layer(chart_co1).configure_view(
    strokeWidth=0
).configure_title(
    fontSize=22,
    font="Arial",
    color="black",
    anchor="start"
).configure_axis(
    labelFont="Arial",
    titleFont="Arial",
    labelFontSize=14,
    titleFontSize=16,
    titleFontWeight="normal",
    labelColor="grey",
    titleColor="grey"
)

alt.LayerChart(...)

In [ ]:
df_plot = df.groupby("Country").sum().sort_values(["TotalPrice"], ascending=False)
df_plot["Country"] = df_plot.index
df_plot["Country"]["United Kingdom"] ="UK"
df_plot["Country"]["EIRE"] ="Ireland"
df_plot["TotalPrice"] = df_plot["TotalPrice"].div(1000)
total = df_plot["TotalPrice"].sum()
uk = df_plot["TotalPrice"]["United Kingdom"]
rest = total - uk
text = [['United Kingdom', uk],['Rest of the world',rest]]
df_plot = pd.DataFrame(text, columns=['Country','TotalPrice'])

#### Chart Beschreibung
Anschließend soll nun dargestellt werden, wie die tatsächliche prozentuale Umsatzverteilung zwischen UK und anderen Ländern bzw. dem Rest der Welt ist, da im vorangegangen Chart bereits zu erkennen war, dass ein sehr großer Anteil auf Uk fällt (Länder neben UK werden zusammengefasst, da die Anteile sehr klein sind - hierdurch wird die Übersichtlichkeit gewährleistet). Als Visualisierungsart wird dabei ein Kreisdiagramm verwendet, um die Verhältnisse der Umsatzverteilung darzustellen. Das Diagramm wird verwendet da nur zwei Ausprägungen vorhanden sind und so eine Interpretation leicht für den Nutzer möglich ist. Um den Chart zu optimieren wird Clutter eleminiert indem Hilfs und Gridlinien, sowie die Rahmen rechts und oben entfernt werden (Closure). Zudem wird die separate Legende entfernt und in den Chart integriert. Hier wird durch den farblichen Text, welcher die zwei Kategorien beschreibt die Legende ersetzt (Closure). Durch die Nähe des Textes an den Kategorien des Chartes(Proximity) werden die Kategorien mit der Schrift zusätzlich assoziiert. Der Fokus wird gezielt mit (hue) Farbe (rot) und (shape) Form, sowie Breite (Width) des Textes auf den aktuellen Stand des Umsatzanteils des Landes UK gelegt. 

In [8]:
# funktioniert
chart = alt.Chart(df_plot).encode(
    theta=alt.Theta("TotalPrice:Q"), 
    color=alt.condition(
        alt.FieldOneOfPredicate("Country", ["United Kingdom"]),
        alt.value("darkred"),     
        alt.value("grey")     
    )
).properties(
    title={"text":["Distribution of revenue between the UK and the rest of the world"], "subtitle":["In the period from 01.12.2010 to 30.11.2011"]},
    width=550,
    height=350
)

pie = chart.mark_arc(outerRadius=130)
#text = chart.mark_text(radius=130, size=12).encode(text="Country:N")

text_uk = alt.Chart().mark_text(
    align="left",
    baseline="bottom",
    fontSize=14,
    fontWeight=400,
    color="darkred"
).encode(
    x=alt.value(35),  # pixels from left
    y=alt.value(50),  # pixels from top
    text=alt.value("United Kingdom (UK)")
)
revenue_uk = alt.Chart().mark_text(
    align="left",
    baseline="bottom",
    fontSize=14,
    fontWeight=700,
    color="darkred"
).encode(
    x=alt.value(35),  # pixels from left
    y=alt.value(70),  # pixels from top
    text=alt.value("Revenue share: 85%")
)

text_rest = alt.Chart().mark_text(
    align="left",
    baseline="bottom",
    fontSize=14,
    fontWeight=400,
    color="grey"
).encode(
    x=alt.value(385),  # pixels from left
    y=alt.value(50),  # pixels from top
    text=alt.value("Rest of the world")
)

revenue_rest = alt.Chart().mark_text(
    align="left",
    baseline="bottom",
    fontSize=14,
    fontWeight=700,
    color="grey"
).encode(
    x=alt.value(385),  # pixels from left
    y=alt.value(70),  # pixels from top
    text=alt.value("Revenue share: 15%")
)

alt.layer(pie, text_uk, revenue_uk, text_rest, revenue_rest).configure_view(
    strokeWidth=0
).configure_title(
    fontSize=22,
    font="Arial",
    color="black",
    anchor="start"
).configure_axis(
    labelFont="Arial",
    titleFont="Arial",
    labelFontSize=14,
    titleFontSize=16
)

alt.LayerChart(...)

In [9]:
df_plot = df
df_plot["InvoiceDate"]= pd.to_datetime(df_plot["InvoiceDate"])
df_plot["Month"] = df_plot["InvoiceDate"].dt.month
df_plot["Year"] = df_plot["InvoiceDate"].dt.year
df_plot = df_plot.groupby(["Month","Year","Country"]).sum()
revenue_uk = 0
revenue_else = 0

for i in df_plot.index:
    if i == "United Kingdom":
        revenue_uk += df_plot.loc[i]["TotalPrice"]
    else:
        revenue_else += df_plot.loc[i]["TotalPrice"]

data = {"Country":[],"Month":[],"Year":[],"Revenue":[]}
for i in df_plot.index:
    revenue_uk = 0
    revenue_rest = 0

    if i[2] == "United Kingdom":
        revenue_uk += df_plot.loc[i]["TotalPrice"]
    else:
        revenue_rest += df_plot.loc[i]["TotalPrice"]

    data["Month"].append(i[0])
    data["Year"].append(i[1])
    data["Country"].append("United Kingdom")
    data["Revenue"].append(revenue_uk)

    data["Month"].append(i[0])
    data["Year"].append(i[1])
    data["Country"].append("Rest of the world")
    data["Revenue"].append(revenue_rest)
df_plot = pd.DataFrame(data)
df_plot = df_plot.groupby(["Country","Month","Year"]).sum()
df_plot = df_plot.reset_index()

df_plot_rest = df_plot[df_plot["Country"]=="Rest of the world"]
df_plot_uk = df_plot[df_plot["Country"]=="United Kingdom"]
df_plot_uk = df_plot_uk.drop(columns=["Country"], axis=1)
df_plot_rest = df_plot_rest.drop(columns=["Country"], axis=1)
df_plot_uk = df_plot_uk.reset_index()
df_plot_uk["Revenue_Share"] = df_plot_uk["Revenue"] / (df_plot_rest["Revenue"] + df_plot_uk["Revenue"])
df_plot = df_plot_uk
df_plot["Day"] = 1
df_plot["Date"] = pd.to_datetime(dict(year=df_plot.Year, month=df_plot.Month,day=df_plot.Day))
df_plot_2 = df_plot.copy()
df_plot_2 = df_plot_2[df_plot_2["Date"] == "2011-11-01"]

#### Chart Beschreibung
Neben der gesamten Umsatzverteilung soll auch der Verlauf dargestellt werden, um mögliche Rückgänge / Zuwächse darzustellen. Als Visualisierungsart wird dabei ein Liniendiagramm verwendet, um das Umsatz-Verhältnis von UK über die Monate darzustellen, da der Umsatzanteil so hoch ist wird nicht der Rest der Welt angezeigt (aufgrund der Skalierung des Graphs - der Abstand der Linien wäre zu groß und würde die leichte Übersicht stören). Durch die Nähe des Textes an dem letzten Datenpunkt des Chartes (Proximity) wird die (Text-)Information mit der Schrift zusätzlich assoziiert. Der Fokus wird gezielt mit (hue) Farbe (rot) auf den Text und somit auf den letzten Stand des Umsatzanteils des Landes UK gelegt. Der Chart nutzt das einheitliche Farbdesign der vorherigen Charts und ist möglichst übersichtlich gestaltet, indem nur die nötigsten Informationen dargestellt werden.

In [10]:
line_chart = alt.Chart(df_plot).mark_line().encode(
    x=alt.X("Date:T",
            axis=alt.Axis(title=None, 
                          labelAngle=0,
                          grid=False,
                          labelExpr="[timeFormat(datum.value, '%b'), timeFormat(datum.value,'%m')=='01' ? timeFormat(datum.value,'%Y'):'']",
                          labelOffset=14,
                          labelPadding=-14,
                          labelAlign="left",
                          tickSize={
                            "condition": {"test": {"field": "value", "timeUnit": "month", "equal": 1}, "value": 20},
                            "value": 20
                            },
                        
                          )),
    y=alt.Y("Revenue_Share:Q",
            scale=alt.Scale(domain=[0, 1]),
            axis=alt.Axis(title = "Revenue share",
                          grid=False,
                          labelAngle=0,
                          titleAnchor="end",
                          values=[0.2,0.4,0.6,0.8,1.0])),
    color=alt.value("grey"),
    strokeWidth=alt.value(4),
).properties(
    title={"text":["Revenue share of United Kingdom"], "subtitle":["In the period from 01.12.2010 to 30.11.2011"]},
    width=550,
    height=350
)

bubble = alt.Chart(df_plot_2).mark_circle(opacity=1, size=150).encode(
    alt.X("Date:T"),
    alt.Y("Revenue_Share:Q"),
    color=alt.value("darkred")
)

label = alt.Chart(df_plot_2).mark_text(align="left", dx=14, size=14, fontWeight="bold", color="darkred").encode(
    alt.X("Date:T", aggregate="max"),
    alt.Y("Revenue_Share:Q", aggregate={"argmax": "Date"}),
    text=alt.value("Current: 88%")
)

alt.layer(line_chart, bubble, label).configure_view(
    strokeWidth=0
).configure_title(
    fontSize=22,
    font="Arial",
    color="black",
    anchor="start"
).configure_axis(
    labelFont="Arial",
    titleFont="Arial",
    labelFontSize=14,
    titleFontSize=16,
    titleFontWeight="normal",
    labelColor="grey",
    titleColor="grey"
)

alt.LayerChart(...)

In [ ]:
df_plot = df.groupby("Country").sum().sort_values(["TotalPrice"], ascending=False)
df_plot["Country"] = df_plot.index
df_plot["Country"]["United Kingdom"] ="UK"
df_plot["Country"]["EIRE"] ="Ireland"
df_plot["TotalPrice"] = df_plot["TotalPrice"].div(1000)
total = df_plot["TotalPrice"].sum()
df_plot["Revenue_Share"] = (df_plot.TotalPrice / total)*100
df_plot = df_plot[:10]
text = [[0, 0, 0, 66 ,'Netherlands', 'In order to reduce the risks of revenue dependency on the United Kingdom in the short-, \n medium- and long-term, different objectives have to be followed. For the short and \n medium term reduction, the target is to reduce the share of sales from the United Kingdom \n to 25 percent in the next 30 months by strengthening sales in regions outside the UK. \n In the long term, each country should only be responsible for ten percent of sales. \n This is to be made possible by a new site opening in the Asia Pacific region by 2030.']]
df_text = pd.DataFrame(text, columns=['Quantity', 'UnitPrice', 'CustomerID', 'TotalPrice','Country','Annotation'])

#### Chart Beschreibung
Nun soll das Ziel kommuniziert werden, die Reduktion des Umsatzanteils von UK durch die Erhöhung der Umsätze in den Ländern außerhalb des Vereinigten Königreichs. Hierfür wird erneut auf die anfängliche Übersicht zurückgegriffen mit der Umsatzverteilung, allerdings nicht mit absoluten Werten sondern prozentualen Umsatzanteil. Wie in dem ersten Graph wird Clutter eleminiert indem Hilfs und Gridlinien, sowie die Rahmen rechts und oben entfernt werden (Closure). Außerdem wird der Fokus gezielt mit (hue) Farbe auf den Umsatz des Landes UK gelegt zudem wird mit (added marks / spatial position)einer Textbeschreibung in der Visualisierung ein Fokus auf die Erklärung der Ziele gelegt, um mögliche Verwirrungen und Unklarheiten zu beseitigen. Im Kontext des Designs wird der Schwerpunkt auf ein schlichtes Design mit wenigen Farben (Schwarz, grau und dunkelrot) gesetzt. Zudem werden Ziellinien hinzugefügt, um die Ziele des Umsatzanteils zu definieren.

In [12]:

# Visualisierung der umsatzstärksten Kunden
chart_co1 = alt.Chart(df_plot).mark_bar().encode(
    y=alt.Y("Revenue_Share",
            axis=alt.Axis(title="Revenue share in percent", 
                          labelAngle=0,
                          titleAnchor="end",
                          grid=False,
                          values=[0,25,50,75,100])),
    x=alt.X("Country",
            sort=["UK", "Netherlands","Ireland","France","Germany","Australia","Spain","Switzerland","Belgium","Sweden"],
            axis=alt.Axis(title = "Country",
                          titleAnchor="start",
                          labelAngle=0)),
    color=alt.condition(
        alt.FieldOneOfPredicate("Country", ["UK"]),
        alt.value("darkred"),     
        alt.value("grey")     
    )
).properties(
    title={"text":["Revenue share by top ten countries with the highest sales"], "subtitle":["In the period from 01.12.2010 to 30.11.2011"]},
    width=650,
    height=350
)

annotations = alt.Chart(df_text).mark_text(
    align='left', 
    baseline='middle',
    dx=8 ,
    lineBreak='\n',
    fontSize=14,
).encode(
    x=alt.X("Country",
            sort=["UK", "Netherlands","Ireland","France","Germany","Australia","Spain","Switzerland","Belgium","Sweden"],
            axis=alt.Axis(title = "Country",
                          titleAnchor="start",
                          labelAngle=0)),
    y='TotalPrice',
    text='Annotation',
)

rule = alt.Chart(
    pd.DataFrame({'y':[10]})
).mark_rule(color='grey', strokeDash=[4, 2]).encode(
    y='y'
)

goal = alt.Chart().mark_text(
    align="left",
    baseline="bottom",
    fontSize=14,
    fontWeight=400,
    color="grey"
).encode(
    x=alt.value(666),  # pixels from left
    y=alt.value(318),  # pixels from top
    text=alt.value("Long-term Goal")
)
rule_short = alt.Chart(
    pd.DataFrame({'y':[25]})
).mark_rule(color='grey', strokeDash=[4, 2]).encode(
    y='y'
)

goal_short = alt.Chart().mark_text(
    align="left",
    baseline="bottom",
    fontSize=14,
    fontWeight=400,
    color="grey"
).encode(
    x=alt.value(666),  # pixels from left
    y=alt.value(257),  # pixels from top
    text=alt.value("Short-term Goal")
)

alt.layer(chart_co1 + rule + goal + rule_short + goal_short + annotations).configure_view(
    strokeWidth=0
).configure_title(
    fontSize=22,
    font="Arial",
    color="black",
    anchor="start"
).configure_axis(
    labelFont="Arial",
    titleFont="Arial",
    labelFontSize=14,
    titleFontSize=16,
    titleFontWeight="normal",
    labelColor="grey",
    titleColor="grey"
)

alt.LayerChart(...)

## H2 Visualizations

In [13]:
purchases = df[~df['InvoiceNo'].str.contains('C')]
returns = df[df['InvoiceNo'].str.contains('C')]
# Delet the numbners of returns out of the purchases
purchases = purchases[~purchases['InvoiceNo'].isin(returns['InvoiceNo'])]
purchases_count = purchases.shape[0]
returns_count = returns.shape[0]
# Create a dataframe with the counts
data = pd.DataFrame({'Category': ['Purchases', 'Returns'], 'Count': [purchases_count, returns_count]})



#### Chart Beschreibung

Um einen klaren Fokus zu schaffen, habe ich den Rahmen des Diagramms entfernt und die Legende durch den Text im Chart ersetzt. Das besonders hervorgehobene Stück ist durch ein leuchtendes Rot hervorgehoben und zieht so die Aufmerksamkeit des Betrachters auf sich. Dabei habe ich bei der Gestaltung auf Einfachheit geachtet und das einheitliche Grau-Rot Farbschema verwendet.

In [14]:
chart = alt.Chart(data).encode(
    theta=alt.Theta("Count:Q"),
    color=alt.condition(
        alt.FieldOneOfPredicate("Category", ["Returns"]),
        alt.value("darkred"),     
        alt.value("grey")     
    )
).properties(
    title={"text":["Distribution of returns versus keept purchases"], "subtitle":["In the period from 01.12.2010 to 30.11.2011"]},
    width=550,
    height=350
)

pie = chart.mark_arc(outerRadius=130,angle=270)

text_returns = alt.Chart().mark_text(
    align="left",
    baseline="bottom",
    fontSize=14,
    fontWeight=700,
    color="darkred"
).encode(
    x=alt.value(55),  # pixels from left
    y=alt.value(173),  # pixels from top
    text=alt.value("Returns: 2%")
)

text_purchases = alt.Chart().mark_text(
    align="left",
    baseline="bottom",
    fontSize=14,
    fontWeight=700,
    color="grey"
).encode(
    x=alt.value(410),  # pixels from left
    y=alt.value(173),  # pixels from top
    text=alt.value("Purchases: 98%")
)

alt.layer(pie, text_returns, text_purchases).configure_view(
    strokeWidth=0
).configure_title(
    fontSize=22,
    font="Arial",
    color="black",
    anchor="start"
).configure_axis(
    labelFont="Arial",
    titleFont="Arial",
    labelFontSize=14,
    titleFontSize=16
)

alt.LayerChart(...)

In [15]:
#df drop Dexcription, quantity, InvoiceDate, UnitPrice ,Country	,TotalPrice
df = df.drop(['Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'Country', 'TotalPrice'], axis=1)
#CustomerID to int
df['CustomerID'] = df['CustomerID'].astype(int)
#Add a cloum with Retunrned and ad a 1 if in InvoiceNo is a C
df['Returned'] = np.where(df['InvoiceNo'].str.contains('C'), 1, 0)
df['Bought'] = np.where(df['InvoiceNo'].str.contains('C'), 0, 1)

#Group by CustomerID and sum the Bought and Returned
df = df.groupby(['CustomerID']).agg({'Bought': 'sum', 'Returned': 'sum'})
#drop all rows where Retundes is bigger than Bought
df = df[df['Returned'] <= df['Bought']]
df['ReturnRate'] =( df['Returned'] / df['Bought'])*100
#df drop rows where Returned is 0
df = df[df['Returned'] != 0]




#### Chart Beschreibung

Um einen klaren Fokus auf die Datenpunkte zu schaffen, habe ich die Achsen leicht ausgegraut und die Labels nah an den Achsen positioniert, um zu verdeutlichen, dass sie zusammengehören. Durch die Entfernung des Rahmens und das Ausblenden des Grids wird die Ablenkung minimiert und der Betrachter kann sich voll und ganz auf die Datenpunkte konzentrieren. Dies erleichtert das Verständnis der Daten und verbessert die visuelle Übersicht.

In [16]:
chart5 = alt.Chart(df).mark_circle().encode(
    x=alt.X('Bought:Q',
             axis=alt.Axis(title='Purchases',
                           titleAnchor='start',
                           labelAngle=0,
                           grid=False)
        ),
    y=alt.Y('ReturnRate:Q',
            axis=alt.Axis(title='Return Rate',
                          titleAnchor='end',
                          labelAngle=0,
                          grid=False,)
                          ),
    color=alt.condition(
        alt.FieldOneOfPredicate("Purchases", [""]),
        alt.value("darkred"),
        alt.value("grey")
    ),
).properties(
    title={"text":["Returnrate compared to number of purchases"], "subtitle":["In the period from 01.12.2010 to 30.11.2011"] },
    width=500,
    height=350
)

alt.layer(chart5).configure_view(
    strokeWidth=0
).configure_title(
    fontSize=22,
    font="Arial",
    color="black",
    anchor="start"
).configure_axis(
    labelFont="Arial",
    titleFont="Arial",
    labelFontSize=14,
    titleFontSize=12,
    titleFontWeight="normal",
    labelColor="grey",
    titleColor="grey"
).transform_filter(
    (alt.datum.Bought < 500) 
)

alt.LayerChart(...)

# Conclusion
Damit die Forschungsfrage „Wie können Abhängigkeiten für den E-Commerce Shop langfristig reduziert werden und welche Maßnahmen können zur langfristigen Gewinnmaximierung getroffen werden?“ beantwortet werden kann werden die zwei Hypothesen „Die Einnahmen des Online-Shops werden maßgeblich vom Land beeinflusst, in dem das Unternehmen ansässig ist“ und „Die Wahrscheinlichkeit, dass ein Kunde ein Produkt zurückgibt, sinkt mit der Anzahl der getätigten Käufe“ genauer betrachtet. Hypothese H1 fokussiert sich auf die Reduktion der Abhängigkeiten, hierbei wird der Zusammenhang zwischen dem Unternehmensstandort (Vereinigten Königreich) und dem Umsatz untersucht. Der Umsatzanteil des Vereinigten Königreiches beträgt 85% somit ist ein signifikanter Anteil des Umsatzes vom VK anhängig. Die große Umsatzabhängigkeit des Unternehmens von einem Land führt zu einem hohen Risiko, da politische Entscheidungen, wirtschaftliche Schwankungen oder sonstige lokale Einflüsse zu einem starken Umsatzrückgang beitragen könnten. Die Hypothese H2 fokussiert sich auf die Maximierung des Gewinns indem diese die Zusammenhänge zwischen Rückgaben und der Häufigkeit der Kundeneinkäufe untersucht. Auffällig ist hierbei, dass die Rückgaben bei einer steigenden Anzahl an Käufen sinkt. Aufgrund der generell niedrigen Retourenrate von rund 2% sind die Retouren kein unternehmensspezifisches Problem (vgl. Repko 2022).
## Limitations and Data issues 
Die Analyse der Hypothese H1 bezieht sich nur auf (Transaktions-)Daten der letzten zwölf Monate, wodurch der begrenzte Zeitraum eine Verzerrung der eigentlichen Umsatztreiber Wiederspiegeln könnte. Deshalb sollten zudem Daten vor den vergangen zwölf Monaten berücksichtigt werden, um keine Fehlentscheidungen zu unterstützen. Außerdem muss beachtet werden, dass keine Unterscheidung zwischen dem Kontext der Großabnehmer gemacht wurde. Es wurde nicht untersucht, ob einzelne Großkunden mit Sitz in UK die Waren auch in anderen Ländern vertreiben, wodurch die direkte Abhängigkeit des Landes sinken könnte. Eine genaue Untersuchung dieser Aspekte wäre (sofern Daten vorhanden sind) zu empfehlen.

Die Hypothese H2 konzentriert sich auf die Retouren der Kunden, dabei werden Produktspezifische Rückgaberaten ignoriert. Dies sollte jedoch nicht isoliert betrachtet werden, da die Retouren insbesondere stark vom Produkt- und der jeweiligen Kategorie abhängig sind. In diesem Kontext sollten Rückgaben auf Produktebene zusätzlich untersucht werden und im Kontext von der Gewinnmarge pro Produkt neu evaluiert werden, um eventuelle Kostenfallen für das Unternehmen zu eliminieren. (Aufgrund der generellen niedrigen Retourenrate und limitierten Datenverfügbarkeit wurde darauf bewusst verzichtet / dem begrenzten Projektumfang).

## Recommended Action
 Das Unternehmen muss die Umsatzabhängigkeit von dem Vereinigten Königreich senken, indem der Umsatz durch eine Marktdurchdringungsstrategie in Ländern außerhalb des VK gestärkt wird. Hierfür werden Investitionen in Marketing und eine Anpassung des Webshops für landesspezifische Präferenzen benötigt werden. Zudem sollte das Unternehmen die Mechanismen zur Verringerung der Retourenrate beibehalten.